# Does everything to run entropy and creates slurm files to submit

Goal is 4 chains k=2-8. 

In [33]:
!source activate py36

/usr/bin/sh: line 0: source: activate: file not found


In [34]:
import sys

#sys.path.append('/home/faske/g/anaconda3/envs/py34/lib/python3.4/site-packages')
sys.path.append('/data/gpfs/assoc/parchmanlab/tfaske/anaconda3/envs/py36/lib/python3.6/site-packages')
sys.path.append("/data/gpfs/assoc/parchmanlab/tfaske/ipynb/include_utils")

import ipyparallel as ipp
import os, time
import include_utils as u
import pandas as pd
import numpy as np
import scipy as sp
import random
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
#import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE, call, check_output
import seaborn as sns
from IPython.display import FileLink
import urllib.request as urllib2
import dill
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
#import pysam
from collections import OrderedDict, namedtuple, Counter
import operator
import multiprocessing as mp
import shutil
import tempfile
#from ipyparallel import Client
import scandir
import glob
from Bio.SeqIO.QualityIO import FastqGeneralIterator
import pickle
import re
from itertools import chain
#import Levenshtein as lv

In [35]:
root = '/data/gpfs/home/tfaske/d/rabbit/full'

In [36]:
cd $root

/data/gpfs/assoc/denovo/tfaske/rabbit/full


### Create mgpl & pntest files with associated dir 

In [5]:
!mkdir entropy

mkdir: cannot create directory ‘entropy’: File exists


In [6]:
ent_dir =os.path.join(root,'entropy')
assert ent_dir

In [7]:
cd $ent_dir

/data/gpfs/assoc/denovo/tfaske/rabbit/full/entropy


In [8]:
!cp '../SNPcall/filtering/good_snps.recode.vcf.gz' .

In [9]:
!gunzip 'good_snps.recode.vcf.gz'

In [10]:
vcf = 'good_snps.recode.vcf'
vcf

'good_snps.recode.vcf'

In [11]:
perl_mpgl = '/data/gpfs/home/tfaske/g/src/perl_scripts/vcf2mpglV1.3TLP.pl'
perl_mean = '/data/gpfs/home/tfaske/g/src/perl_scripts/gl2genestV1.3.pl'

In [12]:
!perl $perl_mpgl $vcf

Number of loci: 32566; number of individuals 585


In [13]:
mpgl = 'good_snps.recode.mpgl'

In [14]:
!perl $perl_mean $mpgl mean

### Create ldak files in R

In [15]:
%load_ext rpy2.ipython

In [16]:
%%R
require(readr)
require(MASS)
require(LEA)
require(ggplot2)

source('/data/gpfs/home/tfaske/g/src/R/Imports.R')

setwd('/data/gpfs/home/tfaske/d/rabbit/full/entropy')

R[write to console]: Loading required package: readr

R[write to console]: Loading required package: MASS

R[write to console]: Loading required package: LEA

R[write to console]: Loading required package: ggplot2



In [17]:
%%R
g <- read.table("pntest_mean_good_snps.recode.txt", header=F)
dim(g)

[1] 32566   585


In [19]:
%%R
#create Pop_ID file 

indv<-read.table("../SNPcall/filtering/good_snps.recode.vcf.012.indv",sep="\t")

Pop <- rep(NA,times=nrow(indv))
ID <- rep(NA,times=nrow(indv))
All <- rep(NA,times=nrow(indv))
for (i in 1:nrow(indv)){
  Pop[i] <- unlist(strsplit(as.character(indv$V1[i]),"_"))[2]
  ID[i] <- unlist(strsplit(as.character(indv$V1[i]),"_"))[3]
  All[i] <- as.character(indv$V1[i])
}
Pop_ID <- data.frame(Pop=Pop,ID=ID,All=All)
head(Pop_ID)
write.csv(Pop_ID,'Pop_ID.csv',row.names=F)

In [20]:
%%R
Pop_ID <- read.csv("Pop_ID.csv")
pca_df <- PCA_entropy(t(g))

    PC1     PC2     PC3     PC4     PC5 
0.06262 0.04122 0.02911 0.02417 0.01536 


In [21]:
%%R
#######################################################################################
                                    #### LDA ####
#######################################################################################

k2<-kmeans(pca_df[,1:5],2,iter.max=10,nstart=10,algorithm="Hartigan-Wong")
k3<-kmeans(pca_df[,1:5],3,iter.max=10,nstart=10,algorithm="Hartigan-Wong")
k4<-kmeans(pca_df[,1:5],4,iter.max=10,nstart=10,algorithm="Hartigan-Wong")
k5<-kmeans(pca_df[,1:5],5,iter.max=10,nstart=10,algorithm="Hartigan-Wong")
k6<-kmeans(pca_df[,1:5],6,iter.max=10,nstart=10,algorithm="Hartigan-Wong")
k7<-kmeans(pca_df[,1:5],7,iter.max=10,nstart=10,algorithm="Hartigan-Wong")
k8<-kmeans(pca_df[,1:5],8,iter.max=10,nstart=10,algorithm="Hartigan-Wong")
k9<-kmeans(pca_df[,1:5],9,iter.max=10,nstart=10,algorithm="Hartigan-Wong")
k10<-kmeans(pca_df[,1:5],10,iter.max=10,nstart=10,algorithm="Hartigan-Wong")

ldak2<-lda(x=pca_df[,1:5],grouping=k2$cluster,CV=TRUE)
ldak3<-lda(x=pca_df[,1:5],grouping=k3$cluster,CV=TRUE)
ldak4<-lda(x=pca_df[,1:5],grouping=k4$cluster,CV=TRUE)
ldak5<-lda(x=pca_df[,1:5],grouping=k5$cluster,CV=TRUE)
ldak6<-lda(x=pca_df[,1:5],grouping=k6$cluster,CV=TRUE)
ldak7<-lda(x=pca_df[,1:5],grouping=k7$cluster,CV=TRUE)
ldak8<-lda(x=pca_df[,1:5],grouping=k8$cluster,CV=TRUE)
ldak9<-lda(x=pca_df[,1:5],grouping=k9$cluster,CV=TRUE)
ldak10<-lda(x=pca_df[,1:5],grouping=k10$cluster,CV=TRUE)

write.table(round(ldak2$posterior,5),file="ldak2.txt",quote=F,row.names=F,col.names=F)
write.table(round(ldak3$posterior,5),file="ldak3.txt",quote=F,row.names=F,col.names=F)
write.table(round(ldak4$posterior,5),file="ldak4.txt",quote=F,row.names=F,col.names=F)
write.table(round(ldak5$posterior,5),file="ldak5.txt",quote=F,row.names=F,col.names=F)
write.table(round(ldak6$posterior,5),file="ldak6.txt",quote=F,row.names=F,col.names=F)                                      
write.table(round(ldak7$posterior,5),file="ldak7.txt",quote=F,row.names=F,col.names=F)
write.table(round(ldak8$posterior,5),file="ldak8.txt",quote=F,row.names=F,col.names=F)
write.table(round(ldak9$posterior,5),file="ldak9.txt",quote=F,row.names=F,col.names=F)
write.table(round(ldak10$posterior,5),file="ldak10.txt",quote=F,row.names=F,col.names=F)

In [22]:
%%R
######### create entropy header ####

Pop_ID_list <- paste(Pop_ID$Pop,Pop_ID$ID,sep='_')

Header <- data.frame(dims = NA,Pop_ID_list)

dim(g)

df <- t(Header)
dims <- paste(dim(g)[2],dim(g)[1],sep = " ")

df[1,1] <- dims

write.table(df,'entropy_header.txt',sep = " ",na ="",
            quote = FALSE,row.names = FALSE,col.names = FALSE)

In [23]:
!head entropy_header.txt

585 32566                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
AH_10 AH_11 AH_12 AH_13 AH_14 AH_15 AH_1 AH_2 AH_3 AH_4 AH_5 AH_6 AH_7 AH_8 AH_9 AR_2 AR_3 AR_4 AR_5 AR_6 AS_16 AS_17 AS_18 AS_19 AS_20 AS_21 AS_22 AS_23 AS_24 AS_25 AS_26 AS_27 AS_28 AS_29 AS_30 BC_10 BC_11 BC_12 BC_13 BC_1 BC_2 BC_3 BC_4 BC_5 BC_6 BC_7 BC_8 BC_9 BH_1 BH_2 BH_3 BH_4 BH_5 BH_6 BL_1 BL_2 BL_3 BL_4 BL_5 BL_6 BM_16 BM_18 BM_19 BM_20 BM_21 BM_22 BM_23 BM_24 BM_25 BM_26 BM_27 BM_28 BM_29 BM_

In [24]:
!cat entropy_header.txt good_snps.recode.mpgl > entropy.mpgl

In [25]:
cd $ent_dir

/data/gpfs/assoc/denovo/tfaske/rabbit/full/entropy


In [26]:
!mkdir shdir

In [27]:
### select slurm options
account = 'cpu-s5-denovo-0'
partition = 'cpu-core-0'
time = '14-00:00:00' #time limit 10
cpus = 1
mem_cpu = 2500
email = 'tfaske@nevada.unr.edu'

#entropy settings
l = 100000
b = 30000
num_k = [2,3,4,5,6,7,8]
chains = 4

In [28]:
def write_entropy_sh(account,partition,time,cpus,mem_cpu,email,l,b,num_k,chains):
    for k in num_k:
        for c in range(1,chains+1):
            jobname = 'k%d_c%d' % (k,c)
            seed = random.randint(1000,9999) #default is clock so messes up if submitted same time 
            with open("shdir/run_entropy_%s.sh" % (jobname), "w") as o:
                o.write("""#!/usr/bin/env bash
#SBATCH --account=%s
#SBATCH --partition=%s
#SBATCH --time=%s
#SBATCH --ntasks 1
#SBATCH --cpus-per-task %d
#SBATCH --mem-per-cpu=%d
#SBATCH --job-name ent_%s
#SBATCH --output output_%s.txt
#SBATCH --mail-type=BEGIN,END
#SBATCH --mail-user=%s\n\n""" % (account,partition,time,cpus,mem_cpu,jobname,jobname,email))
        
                o.write("entropy -i ../entropy.mpgl -o ../entropy_%s.hdf5 -r %d -n 2 -l %d -b %d -t 10 -s 20 -e .01 -k %d -q ../ldak%d.txt -m 1 -w 0"
                        % (jobname,seed,l,b,k,k))
    

In [29]:
write_entropy_sh(account,partition,time,cpus,mem_cpu,email,l,b,num_k,chains)

In [30]:
entShells = !find $ent_dir -name 'run_entropy*.sh'
entShells = entShells.sort()
entShells

['/data/gpfs/home/tfaske/d/rabbit/full/entropy/shdir/run_entropy_k2_c1.sh',
 '/data/gpfs/home/tfaske/d/rabbit/full/entropy/shdir/run_entropy_k2_c2.sh',
 '/data/gpfs/home/tfaske/d/rabbit/full/entropy/shdir/run_entropy_k2_c3.sh',
 '/data/gpfs/home/tfaske/d/rabbit/full/entropy/shdir/run_entropy_k2_c4.sh',
 '/data/gpfs/home/tfaske/d/rabbit/full/entropy/shdir/run_entropy_k3_c1.sh',
 '/data/gpfs/home/tfaske/d/rabbit/full/entropy/shdir/run_entropy_k3_c2.sh',
 '/data/gpfs/home/tfaske/d/rabbit/full/entropy/shdir/run_entropy_k3_c3.sh',
 '/data/gpfs/home/tfaske/d/rabbit/full/entropy/shdir/run_entropy_k3_c4.sh',
 '/data/gpfs/home/tfaske/d/rabbit/full/entropy/shdir/run_entropy_k4_c1.sh',
 '/data/gpfs/home/tfaske/d/rabbit/full/entropy/shdir/run_entropy_k4_c2.sh',
 '/data/gpfs/home/tfaske/d/rabbit/full/entropy/shdir/run_entropy_k4_c3.sh',
 '/data/gpfs/home/tfaske/d/rabbit/full/entropy/shdir/run_entropy_k4_c4.sh',
 '/data/gpfs/home/tfaske/d/rabbit/full/entropy/shdir/run_entropy_k5_c1.sh',
 '/data/gpfs

In [31]:
def write_sbatch_entShells(entShells):
         with open("shdir/sbatch_entropy.sh", "w") as o:
            o.write("""#!/usr/bin/env bash \n""")
            for sh in entShells:
                o.write("""sbatch %s\n""" % (sh))

In [32]:
write_sbatch_entShells(entShells)

# Run all entropy*sh
    cd /data/gpfs/home/tfaske/d/rabbruh/entropy/shdir
    source activate entropy
    bash sbatch_entropy.sh

# Use estpost to get DIC, q, and gprob

In [37]:
np.set_printoptions(precision=8) # increases float print option
pd.set_option("precision", 8)

In [38]:
cd $ent_dir

/data/gpfs/assoc/denovo/tfaske/rabbit/full/entropy


In [39]:
hdf5_files = !find . -name '*hdf5'
hdf5_files = hdf5_files.sort()
hdf5_files

['./entropy_k2_c1.hdf5',
 './entropy_k2_c2.hdf5',
 './entropy_k2_c3.hdf5',
 './entropy_k2_c4.hdf5',
 './entropy_k3_c1.hdf5',
 './entropy_k3_c2.hdf5',
 './entropy_k3_c3.hdf5',
 './entropy_k3_c4.hdf5',
 './entropy_k4_c1.hdf5',
 './entropy_k4_c2.hdf5',
 './entropy_k4_c3.hdf5',
 './entropy_k4_c4.hdf5',
 './entropy_k5_c1.hdf5',
 './entropy_k5_c2.hdf5',
 './entropy_k5_c3.hdf5',
 './entropy_k5_c4.hdf5']

In [40]:
estpost = '/data/gpfs/home/tfaske/g/anaconda3/envs/entropy/bin/estpost.entropy'

In [41]:
#make DIC
for i in range(0,len(hdf5_files)):
    f = hdf5_files[i]
    k = f.split('_')[1] #set this 
    c = f.split('_')[2].split('.hdf5')[0]
    #print(k,c)
    dic = "DIC_%s_%s.txt" % (k,c)
    !$estpost $f -s 3 -p deviance > $dic

In [42]:
dic_files = !find . -name 'DIC*'
dic_files

['./DIC_k2_c4.txt',
 './DIC_k2_c1.txt',
 './DIC_k3_c2.txt',
 './DIC_k4_c4.txt',
 './DIC_k4_c2.txt',
 './DIC_k5_c2.txt',
 './DIC_k4_c3.txt',
 './DIC_k3_c1.txt',
 './DIC_k2_c2.txt',
 './DIC_k5_c4.txt',
 './DIC_k4_c1.txt',
 './DIC_k2_c3.txt',
 './DIC_k3_c4.txt',
 './DIC_k5_c1.txt',
 './DIC_k3_c3.txt',
 './DIC_k5_c3.txt']

In [43]:
for d in dic_files:
    !cat $d
    print('\n')

file = ./entropy_k2_c4.hdf5
parameter dimensions for deviance: samples = 7000, chains = 1
Model deviance: 31937168.19
Effective number of parameters: 16441280.58
Model DIC: 48378448.78


file = ./entropy_k2_c1.hdf5
parameter dimensions for deviance: samples = 7000, chains = 1
Model deviance: 31937094.35
Effective number of parameters: 16806615.33
Model DIC: 48743709.68


file = ./entropy_k3_c2.hdf5
parameter dimensions for deviance: samples = 7000, chains = 1
Model deviance: 30347470.08
Effective number of parameters: 22060478.73
Model DIC: 52407948.81


file = ./entropy_k4_c4.hdf5
parameter dimensions for deviance: samples = 7000, chains = 1
Model deviance: 29389078.44
Effective number of parameters: 24500629.20
Model DIC: 53889707.64


file = ./entropy_k4_c2.hdf5
parameter dimensions for deviance: samples = 7000, chains = 1
Model deviance: 29390247.27
Effective number of parameters: 25826402.87
Model DIC: 55216650.14


file = ./entropy_k5_c2.hdf5
parameter dimensions for deviance: sa

In [44]:
dic_list = []
for d in dic_files:
    k = d.split('_k')[1].split('_')[0] #set this 
    c = d.split('_c')[1].split('.txt')[0]
    #print(k,c)
    
    dic = !grep 'DIC' $d
    dic = float(re.search('(\d+.\d+)',str(dic)).group(0))
    #print(dic)
    
    dic_list.append([k,dic,c])
dic_df = pd.DataFrame(dic_list,columns=['k','DIC','chain'])
dic_df.head()

,k,DIC,chain
0,2,48378448.78,4
1,2,48743709.68,1
2,3,52407948.81,2
3,4,53889707.64,4
4,4,55216650.14,2


In [45]:
dic_df.to_csv('dic_list.csv')

In [46]:
dic_sum = dic_df.groupby('k').describe().DIC

In [47]:
dic_sum.sort_values('mean')

,count,mean,std,min,25%,50%,75%,max
k,,,,,,,,
2,4.0,4.83665595e+07,3.19519236e+05,47962018.15,4.82743411e+07,48380255.110,48472473.5000,48743709.68
3,4.0,5.20820021e+07,5.61304277e+05,51397125.27,5.17522695e+07,52139299.880,52469032.4300,52652283.29
5,4.0,5.29547442e+07,1.38116533e+06,51540139.05,5.20179889e+07,52815137.710,53751893.0450,54648562.52
4,4.0,5.43967747e+07,6.47712719e+05,53865965.69,5.38837722e+07,54252241.575,54765244.1675,55216650.14


In [48]:
dic_sum.to_csv('dic_sum.csv')

### Get gprob and q for each k, and mcmc metrics

In [52]:
#k2
!$estpost *k2*.hdf5 -p q -s 0 -o q2.txt
!$estpost *k2*.hdf5 -p gprob -s 0 -o gprob2.txt

#k3
!$estpost *k3*.hdf5 -p q -s 0 -o q3.txt
!$estpost *k3*.hdf5 -p gprob -s 0 -o gprob3.txt

#k24
!$estpost *k4*.hdf5 -p q -s 0 -o q4.txt
!$estpost *k4*.hdf5 -p gprob -s 0 -o gprob4.txt

#k5
!$estpost *k5*.hdf5 -p q -s 0 -o q5.txt
!$estpost *k5*.hdf5 -p gprob -s 0 -o gprob5.txt

file = entropy_k2_c1.hdf5
file = entropy_k2_c2.hdf5
file = entropy_k2_c3.hdf5
file = entropy_k2_c4.hdf5
parameter dimensions for q: ind = 585, populations = 2, samples = 7000, chains = 4
file = entropy_k2_c1.hdf5
file = entropy_k2_c2.hdf5
file = entropy_k2_c3.hdf5
file = entropy_k2_c4.hdf5
parameter dimensions for gprob: loci = 32566, ind = 585, genotypes = 3, chains = 4
^C
file = entropy_k3_c1.hdf5
file = entropy_k3_c2.hdf5
file = entropy_k3_c3.hdf5
file = entropy_k3_c4.hdf5
parameter dimensions for q: ind = 585, populations = 3, samples = 7000, chains = 4
file = entropy_k3_c1.hdf5
file = entropy_k3_c2.hdf5
file = entropy_k3_c3.hdf5
file = entropy_k3_c4.hdf5
parameter dimensions for gprob: loci = 32566, ind = 585, genotypes = 3, chains = 4
^C
file = entropy_k4_c1.hdf5
file = entropy_k4_c2.hdf5
file = entropy_k4_c3.hdf5
file = entropy_k4_c4.hdf5
parameter dimensions for q: ind = 585, populations = 4, samples = 7000, chains = 4
^C
file = entropy_k4_c1.hdf5
file = entropy_k4_c2.hdf5
file

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/data/gpfs/assoc/parchmanlab/tfaske/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/utils/_process_posix.py", line 156, in system
    child = pexpect.spawn(self.sh, args=['-c', cmd])  # Vanilla Pexpect
  File "/data/gpfs/assoc/parchmanlab/tfaske/anaconda3/envs/py36/lib/python3.6/site-packages/pexpect/pty_spawn.py", line 204, in __init__
    self._spawn(command, args, preexec_fn, dimensions)
  File "/data/gpfs/assoc/parchmanlab/tfaske/anaconda3/envs/py36/lib/python3.6/site-packages/pexpect/pty_spawn.py", line 303, in _spawn
    cwd=self.cwd, **kwargs)
  File "/data/gpfs/assoc/parchmanlab/tfaske/anaconda3/envs/py36/lib/python3.6/site-packages/pexpect/pty_spawn.py", line 314, in _spawnpty
    return ptyprocess.PtyProcess.spawn(args, **kwargs)
  File "/data/gpfs/assoc/parchmanlab/tfaske/anaconda3/envs/py36/lib/python3.6/site-packages/ptyprocess/ptyprocess.py", line 309, in spawn
    exec_err_data = os.read(exec_err_pipe_read, 4096)
Keyboar

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/data/gpfs/assoc/parchmanlab/tfaske/anaconda3/envs/py36/lib/python3.6/site-packages/IPython/utils/_process_posix.py", line 156, in system
    child = pexpect.spawn(self.sh, args=['-c', cmd])  # Vanilla Pexpect
  File "/data/gpfs/assoc/parchmanlab/tfaske/anaconda3/envs/py36/lib/python3.6/site-packages/pexpect/pty_spawn.py", line 204, in __init__
    self._spawn(command, args, preexec_fn, dimensions)
  File "/data/gpfs/assoc/parchmanlab/tfaske/anaconda3/envs/py36/lib/python3.6/site-packages/pexpect/pty_spawn.py", line 303, in _spawn
    cwd=self.cwd, **kwargs)
  File "/data/gpfs/assoc/parchmanlab/tfaske/anaconda3/envs/py36/lib/python3.6/site-packages/pexpect/pty_spawn.py", line 314, in _spawnpty
    return ptyprocess.PtyProcess.spawn(args, **kwargs)
  File "/data/gpfs/assoc/parchmanlab/tfaske/anaconda3/envs/py36/lib/python3.6/site-packages/ptyprocess/ptyprocess.py", line 309, in spawn
    exec_err_data = os.read(exec_err_pipe_read, 4096)
Keyboar

TypeError: must be str, not list

In [54]:
#MCMC diagnostics
!$estpost *k2*.hdf5 -p q -s 4 -o MCMC_k2.txt

!$estpost *k3*.hdf5 -p q -s 4 -o MCMC_k3.txt

!$estpost *k4*.hdf5 -p q -s 4 -o MCMC_k4.txt

!$estpost *k5*.hdf5 -p q -s 4 -o MCMC_k5.txt

file = entropy_k2_c1.hdf5
file = entropy_k2_c2.hdf5
file = entropy_k2_c3.hdf5
file = entropy_k2_c4.hdf5
parameter dimensions for q: ind = 585, populations = 2, samples = 7000, chains = 4
file = entropy_k3_c1.hdf5
file = entropy_k3_c2.hdf5
file = entropy_k3_c3.hdf5
file = entropy_k3_c4.hdf5
parameter dimensions for q: ind = 585, populations = 3, samples = 7000, chains = 4
file = entropy_k4_c1.hdf5
file = entropy_k4_c2.hdf5
file = entropy_k4_c3.hdf5
file = entropy_k4_c4.hdf5
parameter dimensions for q: ind = 585, populations = 4, samples = 7000, chains = 4
file = entropy_k5_c1.hdf5
file = entropy_k5_c2.hdf5
file = entropy_k5_c3.hdf5
file = entropy_k5_c4.hdf5
parameter dimensions for q: ind = 585, populations = 5, samples = 7000, chains = 4
